 Hello everyone, this is my first competition in object detection, in this competition I chose to use the FasterCNN model.

The competition is wheat detection.

I used Ensemble and TTA to improve the prediction result.
Initially the model result without Ensemble and TTA was 0.67.
After using TTA I raised the result to 0.695. After using Ensemble + TTA I raised the result to 0.713

In [ ]:
import sys
sys.path.insert(0, "/kaggle/input/weightedboxesfusion")

import ensemble_boxes

import pandas as pd
import numpy as np
import cv2
import os, re
import gc
import random

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torch.utils.data import DataLoader, Dataset

from matplotlib import pyplot as plt

In [ ]:
DATA_DIR = "/kaggle/input/global-wheat-detection"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
test_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
test_df.shape

# Data Loader For Test

In [ ]:
class WheatDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        records = self.df[self.df['image_id'] == image_id]
    
        if self.transforms:
            sample = {"image": image}
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id


# Load All Model

In [ ]:
def fasterrcnn_resnet50_fpn(path,pretrained_backbone=False):

    backbone = resnet_fpn_backbone('resnet50', pretrained_backbone)
    model = FasterRCNN(backbone, 2)
    model.load_state_dict(torch.load(path))
    model.to(DEVICE)
    model.eval()
    return model

In [ ]:
def get_model_101( path,pretrained=False):
    backbone = resnet_fpn_backbone('resnet101', pretrained=pretrained)
    model = FasterRCNN(backbone, num_classes=2)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    model.load_state_dict(torch.load(path))
    model.to(DEVICE)
    model.eval()
    return model

In [ ]:
model1 =get_model_101("../input/faster-cnn-final-model/fastercnn_resnet_101_145.pth")
model2= get_model_101("../input/fastercnn-155/fastercnn_101_155.pth")
model3= get_model_101("../input/faster-cnn-101/fastercnn_resnet_101_155_WightDecay.pth")

faster_CNN_50_90_2=fasterrcnn_resnet50_fpn("../input/isfix-model/fastercnn_50_fix.pth")

models=[model1,model3,model2,faster_CNN_50_90_2]



# Ensemble Prediction

In [ ]:
from ensemble_boxes import *

def make_ensemble_predictions(images):
    images = list(image.to(DEVICE) for image in images)    
    result = []
    for model in models:
        with torch.no_grad():
            outputs = model(images)
            result.append(outputs)
            del model
            gc.collect()
            torch.cuda.empty_cache()
    return result

def run_wbf_ensemble(predictions, image_index, image_size=1024, iou_thr=0.34, skip_box_thr=0.43, weights=None):
    boxes = [prediction[image_index]['boxes'].data.cpu().numpy()/(image_size-1) for prediction in predictions]
    scores = [prediction[image_index]['scores'].data.cpu().numpy() for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]) for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:
def get_test_transforms():
    return A.Compose([
            ToTensorV2(p=1.0)
        ], p=1.0)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = WheatDataset(test_df, os.path.join(DATA_DIR, "test"), get_test_transforms())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=1,
    drop_last=False,
    collate_fn=collate_fn
)

# Format For CSV File

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

# TTA

In [ ]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = 1024

class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [3,1]] 
        res_boxes[:, [1,3]] = boxes[:, [0,2]]
        return res_boxes

class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

In [ ]:
from itertools import product

tta_transforms = []
for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
    tta_transforms.append(TTACompose([tta_transform for tta_transform in tta_combination if tta_transform]))

In [ ]:
def make_tta_predictions(images, score_threshold=0.57):
    with torch.no_grad():
        images = torch.stack(images).float().to(DEVICE)
        predictions = []
        for tta_transform in tta_transforms:
            result = []
            #ensemble predict
            outputs = make_ensemble_predictions(tta_transform.batch_augment(images.clone()))
           # outputs = model(tta_transform.batch_augment(images.clone()))

            for i, image in enumerate(images):
                #chose the boxes and scores
                boxes, scores, labels = run_wbf_ensemble(outputs, image_index=i)
                #boxes = outputs[i]['boxes'].data.cpu().numpy()   
                #scores = outputs[i]['scores'].data.cpu().numpy()
                
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                boxes = tta_transform.deaugment_boxes(boxes.copy())
                result.append({
                    'boxes': boxes,
                    'scores': scores[indexes],
                })
            predictions.append(result)
    return predictions

In [ ]:
def run_wbf(predictions, image_index, image_size=1024, iou_thr=0.6, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist() for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist() for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).astype(int).tolist() for prediction in predictions]
    boxes, scores, labels = ensemble_boxes.ensemble_boxes_wbf.weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:
results = []
list_box=[]
list_score=[]
list_image_id=[]
for images, image_ids in test_data_loader:

    predictions = make_tta_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = boxes.round().astype(np.int32).clip(min=0, max=1023)
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        list_score.append(scores)
        list_box.append(boxes)
        list_image_id.append(image_id)
        
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)

In [ ]:
plt.figure(figsize=[30,30])
for i in range(len(list_box)):
    image=plt.imread('../input/global-wheat-detection/test/'+list_image_id[i]+'.jpg')
  
    for b,s in zip(list_box[i],list_score[i]):
            image = cv2.rectangle(image, (b[0],b[1]), (b[0]+b[2],b[1]+b[3]), (255,0,0), 2) 
            image = cv2.putText(image, '{:.2}'.format(s), (b[0]+np.random.randint(3),b[1]), cv2.FONT_HERSHEY_SIMPLEX,  
                           1, (0,255,0), 2, cv2.LINE_AA)
    plt.subplot(4,3,i+1)
    plt.imshow(image)
plt.show()

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df

In [ ]:
test_df.to_csv('submission.csv', index=False)